In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex客户端库：用于在线预测的自定义训练图像分类模型与解释

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_custom_image_classification_online_explain.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_custom_image_classification_online_explain.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
</table>
<br/><br/><br/>

## 概览

本教程演示了如何使用Python的Vertex客户端库来训练和部署自定义图像分类模型，以进行在线预测并进行解释。

数据集

本教程使用的数据集是来自[TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview)的[CIFAR10数据集](https://www.tensorflow.org/datasets/catalog/cifar10)。你将使用的数据集版本已经内置在TensorFlow中。训练模型预测图像属于十个类别中的哪一类：飞机、汽车、鸟、猫、鹿、狗、青蛙、马、船、卡车。

### 目标

在本教程中，您将使用Vertex客户端库在Google预构建的Docker容器中从Python脚本创建一个自定义模型，然后通过发送数据对部署的模型进行预测和解释。您也可以选择使用`gcloud`命令行工具或在Google Cloud控制台上在线创建自定义模型。

执行的步骤包括：

- 创建一个用于训练模型的Vertex自定义作业。
- 训练一个TensorFlow模型。
- 检索和加载模型制品。
- 查看模型评估。
- 设置解释参数。
- 将模型上传为Vertex `Model`资源。
- 将`Model`资源部署到一个服务`Endpoint`资源。
- 进行带解释的预测。
- 取消部署`Model`资源。

### 成本

本教程使用 Google Cloud（GCP）的可计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 来根据您的预期使用情况生成成本估算。

安装

安装最新版本的Vertex客户端库。

In [ ]:
import os
import sys

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install -U google-cloud-aiplatform $USER_FLAG

安装google-cloud-storage库的最新GA版本。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

In [ ]:
if os.environ["IS_TESTING"]:
    ! pip3 install -U tensorflow

In [ ]:
if os.environ["IS_TESTING"]:
    ! pip3 install -U opencv-python

重启内核

一旦您安装了Vertex客户端库和Google *云存储*，您需要重新启动笔记本内核，这样它才能找到这些软件包。

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU 运行时

*如果有选择的话，请确保在 GPU 运行时下运行此笔记本。在 Colab 中，选择* **运行时 > 更改运行时类型 > GPU**

### 设置您的谷歌云项目

**无论您的笔记本环境如何，下面的步骤都是必需的。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您会获得$300 的免费计算/存储成本信用额度。

2. [确保您的项目已启用计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用 Vertex API 和 Compute Engine API。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [谷歌云 SDK](https://cloud.google.com/sdk) 已经安装在谷歌云笔记本中。

5. 在下面的单元格中输入您的项目 ID。然后运行该单元格，确保 Cloud SDK 在此笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter 运行具有 `!` 前缀的行作为 shell 命令，并将带有 `$` 前缀的 Python 变量插入这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改“REGION”变量，该变量用于整个笔记本其余部分的操作。以下是Vertex支持的区域。我们建议您选择离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能不会使用多区域存储桶来训练Vertex。并非所有区域都支持所有Vertex服务。有关每个区域的最新支持，请参阅[Vertex位置文档](https://cloud.google.com/vertex-ai/docs/general/locations)

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您参加的是现场教程会话，您可能正在使用共享的测试账户或项目。为了避免在创建的资源上发生名称冲突，您可以为每个实例会话创建一个时间戳，并附加到在本教程中将要创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的Google云帐户

**如果您正在使用Google云笔记本**，则您的环境已经得到验证。请跳过此步骤。

**如果您正在使用Colab**，运行下面的单元格，并在提示时按照说明通过oAuth验证您的帐户。

**否则**，请按照以下步骤操作：

在 Cloud 控制台中，转到[创建服务账户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务账户**。

在**服务账户名称**字段中输入一个名称，然后点击**创建**。

在**授予此服务账户访问项目的权限**部分，点击角色下拉列表。在筛选框中输入"Vertex"，并选择**Vertex管理员**。在筛选框中输入"Storage Object Admin"，并选择**存储对象管理员**。

点击创建。包含您的密钥的JSON文件将下载到您的本地环境。

在下面的单元格中将您的服务账户密钥路径输入为GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用何种笔记本环境，以下步骤都是必需的。**

当您使用 Vertex 客户端库提交自定义训练作业时，您需要将包含训练代码的 Python 包上传到云存储桶中。Vertex 将从此包中运行代码。在本教程中，Vertex 还会将您作业的训练模型保存在同一个存储桶中。您可以基于这个输出创建一个 `Endpoint` 资源，以便提供在线预测。

在下面设置您的云存储桶的名称。存储桶的名称必须在所有 Google Cloud 项目中唯一，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有在您的存储桶尚不存在时才能运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查内容来验证对云存储桶的访问权限。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

#### 导入Vertex客户端库

将Vertex客户端库导入到我们的Python环境中。

In [ ]:
import time

import google.cloud.aiplatform_v1beta1 as aip
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

顶点常量

为顶点设置以下常量：

- `API_ENDPOINT`：顶点 API 服务的数据集、模型、作业、流水线和端点服务的端点。
- `PARENT`：顶点的位置根路径，用于数据集、模型、作业、流水线和端点资源。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

#### 硬件加速器

设置硬件加速器（例如 GPU）进行训练和预测。

设置变量`TRAIN_GPU/TRAIN_NGPU`和`DEPLOY_GPU/DEPLOY_NGPU`，以使用支持 GPU 的容器映像，并指定分配给虚拟机（VM）实例的 GPU 数量。例如，要使用一个每个 VM 分配了 4 个 Nvidia Telsa K80 GPU 的 GPU 容器映像，您可以指定：

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)

对于 GPU，可用的加速器包括：
   - aip.AcceleratorType.NVIDIA_TESLA_K80
   - aip.AcceleratorType.NVIDIA_TESLA_P100
   - aip.AcceleratorType.NVIDIA_TESLA_P4
   - aip.AcceleratorType.NVIDIA_TESLA_T4
   - aip.AcceleratorType.NVIDIA_TESLA_V100

否则，指定`(None, None)`来使用一个在 CPU 上运行的容器映像。

*注意*：TF 2.3 之前的版本对 GPU 支持会导致在本教程中加载自定义模型时失败。这是个已知问题，在 TF 2.3 中得到解决 -- 这是由生成在服务函数中的静态图运算导致的。如果您在自定义模型中遇到此问题，请使用支持 GPU 的 TF 2.3 容器映像。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (aip.AcceleratorType.NVIDIA_TESLA_K80, 1)

if os.getenv("IS_TESTING_DEPOLY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPOLY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

#### 容器（Docker）映像

接下来，我们将设置用于训练和预测的Docker容器映像

- TensorFlow 1.15
   - `gcr.io/cloud-aiplatform/training/tf-cpu.1-15:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.1-15:latest`
- TensorFlow 2.1
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-1:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-1:latest`
- TensorFlow 2.2
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-2:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-2:latest`
- TensorFlow 2.3
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-3:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-3:latest`
- TensorFlow 2.4
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-4:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-4:latest`
- XGBoost
   - `gcr.io/cloud-aiplatform/training/xgboost-cpu.1-1`
- Scikit-learn
   - `gcr.io/cloud-aiplatform/training/scikit-learn-cpu.0-23:latest`
- Pytorch
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-4:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-5:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-6:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-7:latest`

有关最新列表，请查看[用于训练的预构建容器](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers)。

- TensorFlow 1.15
   - `gcr.io/cloud-aiplatform/prediction/tf-cpu.1-15:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf-gpu.1-15:latest`
- TensorFlow 2.1
   - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-1:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-1:latest`
- TensorFlow 2.2
   - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-2:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-2:latest`
- TensorFlow 2.3
   - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-3:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-3:latest`
- XGBoost
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.1-2:latest`
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.1-1:latest`
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.0-90:latest`
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.0-82:latest`
- Scikit-learn
   - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-23:latest`
   - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-22:latest`
   - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-20:latest`

有关最新列表，请查看[用于预测的预构建容器](https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2-1"

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

TRAIN_IMAGE = "gcr.io/cloud-aiplatform/training/{}:latest".format(TRAIN_VERSION)
DEPLOY_IMAGE = "gcr.io/cloud-aiplatform/prediction/{}:latest".format(DEPLOY_VERSION)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)
print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量`TRAIN_COMPUTE`和`DEPLOY_COMPUTE`来配置用于训练和预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB内存。
     - `n1-highmem`：每个vCPU 6.5GB内存
     - `n1-highcpu`：每个vCPU 0.9GB内存
 - `vCPUs`：数量为\[2, 4, 8, 16, 32, 64, 96\]

*注意：以下情况不支持用于训练：*

 - `standard`：2个vCPU
 - `highcpu`：2个、4个和8个vCPU

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

教程

现在您可以开始创建您自己的自定义模型，并为CIFAR10数据集进行训练。

## 设置客户端

Vertex客户端库按照客户端/服务器模型运行。在您的一侧（Python脚本）上，您将创建一个客户端，用于向Vertex服务器发送请求并接收响应。

在本教程中，您将为工作流程中的不同步骤使用不同的客户端。因此，请提前设置好它们。

- `Model`资源的模型服务。
- 部署的端点服务。
- 用于批处理作业和自定义训练的作业服务。
- 用于提供服务的预测服务。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_job_client():
    client = aip.JobServiceClient(client_options=client_options)
    return client


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_endpoint_client():
    client = aip.EndpointServiceClient(client_options=client_options)
    return client


def create_prediction_client():
    client = aip.PredictionServiceClient(client_options=client_options)
    return client


clients = {}
clients["job"] = create_job_client()
clients["model"] = create_model_client()
clients["endpoint"] = create_endpoint_client()
clients["prediction"] = create_prediction_client()

for client in clients.items():
    print(client)

## 训练模型

您可以使用容器映像的两种方式来训练自定义模型：

- **使用Google Cloud预构建容器**。如果您使用预构建的容器，您还需要指定一个要安装到容器映像中的Python软件包。这个Python软件包包含了您训练自定义模型的代码。

- **使用您自己的自定义容器映像**。如果您使用自己的容器，那么容器需要包含您训练自定义模型的代码。

## 准备您的自定义工作规范

现在您的客户已经准备好了，您的第一步是为您的自定义培训工作创建一个工作规范。工作规范将包括以下内容：

- `worker_pool_spec`：您将用于培训的机器类型的规范以及使用的数量（单个或分布式）
- `python_package_spec`：要与预构建容器一起安装的Python包的规范。

### 准备您的机器规格

现在为您的定制训练作业定义机器规格。这将告诉Vertex需要为训练提供什么类型的机器实例。
- `machine_type`：要提供的GCP实例的类型，例如 n1-standard-8。
- `accelerator_type`：硬件加速器的类型，如果有的话。在本教程中，如果您之前设置了变量`TRAIN_GPU != None`，则使用GPU；否则将使用CPU。
- `accelerator_count`：加速器的数量。

In [ ]:
if TRAIN_GPU:
    machine_spec = {
        "machine_type": TRAIN_COMPUTE,
        "accelerator_type": TRAIN_GPU,
        "accelerator_count": TRAIN_NGPU,
    }
else:
    machine_spec = {"machine_type": TRAIN_COMPUTE, "accelerator_count": 0}

### 准备您的磁盘规格

（可选）现在为您的自定义训练作业定义磁盘规格。这将告诉 Vertex 在每台机器实例中为训练提供什么类型和大小的磁盘。

- `boot_disk_type`：SSD 或标准。SSD 更快，标准更便宜。默认为 SSD。
- `boot_disk_size_gb`：磁盘的大小（以 GB 为单位）。

In [ ]:
DISK_TYPE = "pd-ssd"  # [ pd-ssd, pd-standard]
DISK_SIZE = 200  # GB

disk_spec = {"boot_disk_type": DISK_TYPE, "boot_disk_size_gb": DISK_SIZE}

定义工作池规范

接下来，您需要定义自定义训练作业的工作池规范。工作池规范将包括以下内容：

- `replica_count`: 要提供的此机器类型的实例数。
- `machine_spec`: 硬件规范。
- `disk_spec` : (可选) 磁盘存储规范。

- `python_package`: 要安装在VM实例上的Python训练包，以及要调用的Python模块，以及Python模块的命令行参数。

现在让我们深入了解python包规范：

- `executor_image_spec`: 这是针对您自定义训练作业配置的docker镜像。

- `package_uris`: 这是要安装在已提供实例上的Python训练包的位置（URIs）的列表。这些位置需要在云存储存储桶中。这些可以是单独的Python文件，也可以是整个包的zip（归档）。在后一种情况下，作业服务将将内容解压缩（解存档）到docker镜像中。

- `python_module`: 要调用以运行自定义训练作业的Python模块（脚本）。在本例中，您将调用 `trainer.task.py` -- 请注意，不需要附加 `.py` 后缀。

- `args`: 要传递给相应Python模块的命令行参数。在此示例中，您将设置:
  - `"--model-dir=" + MODEL_DIR`: 存储模型工件的云存储位置。有两种方法告诉训练脚本在哪里保存模型工件：
     - 直接: 将云存储位置作为命令行参数传递给您的培训脚本（设置变量 `DIRECT = True`），
     - 间接: 服务将模型工件位置作为环境变量 `AIP_MODEL_DIR` 传递给您的训练脚本（设置变量 `DIRECT = False`）。在这种情况下，您告诉服务作业规范中的模型工件位置。
  - `"--epochs=" + EPOCHS`: 训练的时代数量。
  - `"--steps=" + STEPS`: 每个时代的步数（批次）数量。
  - `"--distribute=" + TRAIN_STRATEGY"` : 用于单一或分布式训练的训练分布策略。
     - `"single"`: 单个设备。
     - `"mirror"`: 单个计算实例上的所有GPU设备。
     - `"multi"`: 所有计算实例上的所有GPU设备。

In [ ]:
JOB_NAME = "custom_job_" + TIMESTAMP
MODEL_DIR = "{}/{}".format(BUCKET_NAME, JOB_NAME)

if not TRAIN_NGPU or TRAIN_NGPU < 2:
    TRAIN_STRATEGY = "single"
else:
    TRAIN_STRATEGY = "mirror"

EPOCHS = 20
STEPS = 100

DIRECT = True
if DIRECT:
    CMDARGS = [
        "--model-dir=" + MODEL_DIR,
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
        "--distribute=" + TRAIN_STRATEGY,
    ]
else:
    CMDARGS = [
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
        "--distribute=" + TRAIN_STRATEGY,
    ]

worker_pool_spec = [
    {
        "replica_count": 1,
        "machine_spec": machine_spec,
        "disk_spec": disk_spec,
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": [BUCKET_NAME + "/trainer_cifar10.tar.gz"],
            "python_module": "trainer.task",
            "args": CMDARGS,
        },
    }
]

### 拼接工作规范

现在组合定制工作规范的完整描述：

- `display_name`：您为该自定义工作指定的可读名称。
- `job_spec`：自定义工作的规范。
   - `worker_pool_specs`：用于机器VM实例的规范。
   - `base_output_directory`：这告诉服务在哪里保存模型工件的云存储位置（当变量`DIRECT = False`时）。 然后服务将将位置传递给训练脚本作为环境变量`AIP_MODEL_DIR`，路径将为以下形式：

                <output_uri_prefix>/model

In [ ]:
if DIRECT:
    job_spec = {"worker_pool_specs": worker_pool_spec}
else:
    job_spec = {
        "worker_pool_specs": worker_pool_spec,
        "base_output_directory": {"output_uri_prefix": MODEL_DIR},
    }

custom_job = {"display_name": JOB_NAME, "job_spec": job_spec}

### 检查培训包

#### 包布局

在开始培训之前，您将查看如何为自定义培训任务组装 Python 包。解压缩后，包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件 `setup.cfg` 和 `setup.py` 是安装包到 Docker 镜像的操作环境的指令。

文件 `trainer/task.py` 是执行自定义培训任务的 Python 脚本。*注意*，当我们在工作进程池规范中引用它时，我们将目录斜杠替换为点 (`trainer.task`) 并去掉文件后缀 (`.py`)。

#### 包组装

在以下单元格中，您将组装培训包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: CIFAR10 image classification\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### Task.py 内容

在下一个单元格中，您将编写训练脚本 task.py 的内容。我们不会详细讨论，只是让您浏览一下。总结如下：

- 从命令行 (`--model_dir`) 或者环境变量 `AIP_MODEL_DIR` 中获取要保存模型文件的目录。
- 使用 TF Datasets (tfds) 加载 CIFAR10 数据集。
- 使用 TF.Keras 模型 API 构建模型。
- 编译模型 (`compile()`).
- 根据参数 `args.distribute` 设置训练分布策略。
- 使用参数 `args.epochs` 和 `args.steps` 训练模型 (`fit()`).
- 将训练好的模型保存到指定的模型目录中 (`save(args.model_dir)`).

In [ ]:
%%writefile custom/trainer/task.py
# Single, Mirror and Multi-Machine Distributed Training for CIFAR-10

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv("AIP_MODEL_DIR"), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.01, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=10, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=200, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))
print('DEVICES', device_lib.list_local_devices())

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))

# Preparing dataset
BUFFER_SIZE = 10000
BATCH_SIZE = 64


def make_datasets_unbatched():

  # Scaling CIFAR10 data from (0, 255] to (0., 1.]
  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.0
    return image, label


  datasets, info = tfds.load(name='cifar10',
                            with_info=True,
                            as_supervised=True)
  return datasets['train'].map(scale).cache().shuffle(BUFFER_SIZE).repeat()


# Build the Keras model
def build_and_compile_cnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(32, 32, 3)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Conv2D(32, 3, activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(10, activation='softmax')
  ])
  model.compile(
      loss=tf.keras.losses.sparse_categorical_crossentropy,
      optimizer=tf.keras.optimizers.SGD(learning_rate=args.lr),
      metrics=['accuracy'])
  return model


# Train the model
NUM_WORKERS = strategy.num_replicas_in_sync
# Here the batch size scales up by number of workers since
# `tf.data.Dataset.batch` expects the global batch size.
GLOBAL_BATCH_SIZE = BATCH_SIZE * NUM_WORKERS
train_dataset = make_datasets_unbatched().batch(GLOBAL_BATCH_SIZE)

with strategy.scope():
  # Creation of dataset, and model building/compiling need to be within
  # `strategy.scope()`.
  model = build_and_compile_cnn_model()

model.fit(x=train_dataset, epochs=args.epochs, steps_per_epoch=args.steps)
model.save(args.model_dir)

### 将培训脚本存储在您的云存储存储桶中

接下来，您将培训文件夹打包成压缩的tar文件，并将其存储在您的云存储存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_NAME/trainer_cifar10.tar.gz

### 训练模型

现在开始在Vertex上训练您的自定义训练作业。使用此辅助函数`create_custom_job`，它接受以下参数：

- `custom_job`：自定义作业的规范。

该辅助函数调用作业客户端服务的`create_custom_job`方法，并带有以下参数：

- `parent`：指向`Dataset`，`Model`和`Endpoint`资源的Vertex位置路径。
- `custom_job`：自定义作业的规范。

您将显示在`response`对象中返回的一些字段，其中最感兴趣的两个字段是：

`response.name`：分配给此自定义训练作业的Vertex完全限定标识符。您将保存此标识符以在后续步骤中使用。

`response.state`：自定义训练作业的当前状态。

In [ ]:
def create_custom_job(custom_job):
    response = clients["job"].create_custom_job(parent=PARENT, custom_job=custom_job)
    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = create_custom_job(custom_job)

现在获取您创建的自定义作业的唯一标识符。

In [ ]:
# The full unique ID for the custom job
job_id = response.name
# The short numeric ID for the custom job
job_short_id = job_id.split("/")[-1]

print(job_id)

获取自定义作业的信息

接下来，使用以下辅助函数`get_custom_job`，该函数接受以下参数：

- `name`: 自定义作业的 Vertex 完全限定标识符。

辅助函数调用作业客户端服务的`get_custom_job`方法，参数如下：

- `name`: 自定义作业的 Vertex 完全限定标识符。

如果你还记得，当调用`create_custom_job`方法时，在`response.name`字段中获取了自定义作业的 Vertex 完全限定标识符，并将该标识符保存在变量`job_id`中。

In [ ]:
def get_custom_job(name, silent=False):
    response = clients["job"].get_custom_job(name=name)
    if silent:
        return response

    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = get_custom_job(job_id)

# 部署

训练上述模型可能需要超过20分钟的时间。

一旦您的模型训练完成，您可以通过将`end_time`减去`start_time`来计算训练模型实际花费的时间。对于您的模型，我们需要知道保存模型的位置，即Python脚本保存在您本地云存储桶中的`MODEL_DIR + '/saved_model.pb'`。

In [ ]:
while True:
    response = get_custom_job(job_id, True)
    if response.state != aip.JobState.JOB_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        model_path_to_deploy = None
        if response.state == aip.JobState.JOB_STATE_FAILED:
            break
    else:
        if not DIRECT:
            MODEL_DIR = MODEL_DIR + "/model"
        model_path_to_deploy = MODEL_DIR
        print("Training Time:", response.update_time - response.create_time)
        break
    time.sleep(60)

print("model_to_deploy:", model_path_to_deploy)

加载保存的模型

您的模型以 TensorFlow SavedModel 格式保存在 Cloud Storage 存储桶中。现在从 Cloud Storage 存储桶加载它，然后您可以做一些事情，比如评估模型并进行预测。

要加载模型，您可以使用 TF.Keras 的 `model.load_model()` 方法，并向其传递保存模型的 Cloud Storage 路径 -- 由 `MODEL_DIR` 指定。

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model(MODEL_DIR)

## 评估模型

现在来了解一下模型的好坏。

### 载入评估数据

您将从`tf.keras.datasets`中载入CIFAR10测试（留存）数据，使用`load_data()`方法。这将返回一个由两个元素组成的元组数据集。第一个元素是训练数据，第二个是测试数据。每个元素也是一个由两个元素组成的元组：图像数据和相应的标签。

您不需要训练数据，这就是为什么我们将其载入为`(_, _)`。

在您对数据进行评估之前，您需要进行预处理：

x_test:
1. 通过将每个像素除以255来对像素数据进行归一化（重新缩放）。这将用0到1之间的32位浮点数替换每一个单字节整数像素。

y_test:
2. 标签目前是标量（稀疏）。如果您回顾`trainer/task.py`脚本中的`compile()`步骤，您会发现它是为稀疏标签编译的。因此我们不需要进行其他操作。

In [ ]:
import numpy as np
from tensorflow.keras.datasets import cifar10

(_, _), (x_test, y_test) = cifar10.load_data()
x_test = (x_test / 255.0).astype(np.float32)

print(x_test.shape, y_test.shape)

### 执行模型评估

现在评估一下定制工作中的模型表现如何。

In [ ]:
model.evaluate(x_test, y_test)

## 上传模型进行服务

接下来，您将把来自自定义作业的TF.Keras模型上传到Vertex `Model`服务，这将为您的自定义模型创建一个Vertex `Model`资源。在上传过程中，您需要定义一个服务函数来将数据转换为模型期望的格式。如果您向Vertex发送编码数据，您的服务函数将确保在传递给模型输入之前，在模型服务器上对数据进行解码。

### 服务函数如何工作

当您向在线预测服务器发送请求时，请求会被一个HTTP服务器接收。HTTP服务器会从HTTP请求内容正文中提取预测请求。提取的预测请求会被转发到服务函数。对于谷歌预构建的预测容器，请求内容会作为一个`tf.string`传递给服务函数。

服务函数由两部分组成：

- `预处理函数`：
  - 将输入（`tf.string`）转换为底层模型（动态图）的输入形状和数据类型。
  - 执行与训练底层模型期间相同的数据预处理 -- 例如，标准化、缩放等。
  
- `后处理函数`：
  - 将模型输出转换为接收应用程序期望的格式 -- 例如，压缩输出。
  - 打包输出以供接收应用程序使用 -- 例如，添加标题，制作JSON对象等。

预处理和后处理函数都会被转换为与模型融合的静态图。来自底层模型的输出会传递给后处理函数。后处理函数会将转换/打包的输出传递回HTTP服务器。HTTP服务器会将输出作为HTTP响应内容返回给客户端。

在为TF.Keras模型构建服务函数时，需要考虑一个因素，即它们以静态图的形式运行。这意味着您不能使用需要动态图的TF图操作。如果您这样做，您在编译服务函数时将会出错，并指出您正在使用不受支持的EagerTensor。

### 图像数据的服务功能

要将图像传递给预测服务，您需要将压缩（例如JPEG）图像字节编码为base 64——这样可以使内容在通过网络传输二进制数据时免受修改。由于该部署模型期望输入数据为原始（未压缩）字节，您需要确保base 64编码的数据在传递给部署模型之前被转换回原始字节。

为了解决这个问题，定义一个服务函数（`serving_fn`）并将其附加到模型作为预处理步骤。添加一个`@tf.function`装饰器，以便将服务函数融合到底层模型中（而不是在CPU上游）。

当您发送预测或解释请求时，请求的内容通过base 64解码为一个Tensorflow字符串（`tf.string`），然后传递给服务函数（`serving_fn`）。服务函数将`tf.string`预处理为原始（未压缩）的numpy字节（`preprocess_fn`）以匹配模型的输入要求：
- `io.decode_jpeg`- 解压缩JPG图像，返回一个具有三个通道（RGB）的Tensorflow张量。
- `image.convert_image_dtype` - 将整数像素值转换为float 32，并重新调整像素数据在0和1之间。
- `image.resize` - 将图像调整大小以匹配模型的输入形状。

在这一点上，数据可以传递给模型 (`m_call`)。

#### XAI 签名

当服务函数与底层模型一起保存时（`tf.saved_model.save`），您需要将服务函数的输入层指定为签名`service_default`。

对于XAI图像模型，您需要从服务函数保存两个附加的签名：

- `xai_preprocess`：服务函数中的预处理函数。
- `xai_model`：调用模型的具体函数。

In [ ]:
CONCRETE_INPUT = "numpy_inputs"


def _preprocess(bytes_input):
    decoded = tf.io.decode_jpeg(bytes_input, channels=3)
    decoded = tf.image.convert_image_dtype(decoded, tf.float32)
    resized = tf.image.resize(decoded, size=(32, 32))
    return resized


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def preprocess_fn(bytes_inputs):
    decoded_images = tf.map_fn(
        _preprocess, bytes_inputs, dtype=tf.float32, back_prop=False
    )
    return {
        CONCRETE_INPUT: decoded_images
    }  # User needs to make sure the key matches model's input


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def serving_fn(bytes_inputs):
    images = preprocess_fn(bytes_inputs)
    prob = m_call(**images)
    return prob


m_call = tf.function(model.call).get_concrete_function(
    [tf.TensorSpec(shape=[None, 32, 32, 3], dtype=tf.float32, name=CONCRETE_INPUT)]
)

tf.saved_model.save(
    model,
    model_path_to_deploy,
    signatures={
        "serving_default": serving_fn,
        # Required for XAI
        "xai_preprocess": preprocess_fn,
        "xai_model": m_call,
    },
)

获取服务函数签名

您可以通过重新加载模型到内存中，并查询每个层对应的签名来获取模型的输入和输出层的签名。

在进行预测请求时，您需要将请求路由到服务函数而不是模型，因此您需要了解服务函数的输入层名称 -- 这将在稍后进行预测请求时使用。

您还需要知道服务函数的输入和输出层的名称，以构建解释元数据 -- 这将在接下来讨论。

In [ ]:
loaded = tf.saved_model.load(model_path_to_deploy)

serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)
serving_output = list(loaded.signatures["serving_default"].structured_outputs.keys())[0]
print("Serving function output:", serving_output)

input_name = model.input.name
print("Model input name:", input_name)
output_name = model.output.name
print("Model output name:", output_name)

### 解释规范

在进行预测时获得解释，您必须启用解释功能，并在将自定义模型上传到 Vertex `Model` 资源时设置相应的设置。这些设置被称为解释元数据，包括：

- `parameters`：这是用于对模型进行解释的解释算法的规范。您可以选择以下选项之一：
  - Shapley - *注意*，不推荐用于图像数据 -- 可能会运行很长时间
  - XRAI
  - 综合梯度

- `metadata`：这是应用于您的自定义模型的算法规范。

#### 解释参数

让我们首先深入探讨解释算法的设置。

#### Shapley

将结果归因于每个特征，并考虑特征的不同排列。该方法提供了精确 Shapley 值的采样近似。

用例：
  - 在表格数据上的分类和回归。

参数：

- `path_count`：这是算法要处理的特征路径数量。精确近似 Shapley 值需要 M! 条路径，其中 M 是特征数量。对于 CIFAR10 数据集，这将是 784（28*28）。

对于任何数量的特征，这都是计算代价太高。您可以将路径数量减少到 M * `path_count`。

#### 综合梯度

一种基于梯度的方法，用于高效计算具有与 Shapley 值相同公设属性的特征归因。

用例：
  - 在表格数据上的分类和回归。
  - 图像数据上的分类。

参数：

- `step_count`：这是用于近似剩余总和的步骤数量。步数越多，积分近似越准确。一般的经验法则是 50 步，但随着步数增加，计算时间也会增加。

#### XRAI

基于综合梯度方法，XRAI 评估图像的重叠区域，创建一个显著性地图，可以突出图像的相关区域而不是像素。

用例：

  - 图像数据上的分类。

参数：

- `step_count`：这是用于近似剩余总和的步骤数量。步数越多，积分近似越准确。一般的经验法则是 50 步，但随着步数增加，计算时间也会增加。

在下一个代码单元格中，将变量 `XAI` 设置为您将在自定义模型上使用的解释算法。

In [ ]:
XAI = "ig"  # [ shapley, ig, xrai ]

if XAI == "shapley":
    PARAMETERS = {"sampled_shapley_attribution": {"path_count": 10}}
elif XAI == "ig":
    PARAMETERS = {"integrated_gradients_attribution": {"step_count": 50}}
elif XAI == "xrai":
    PARAMETERS = {"xrai_attribution": {"step_count": 50}}

parameters = aip.ExplanationParameters(PARAMETERS)

#### 解释元数据

让我们首先深入了解解释元数据，它包括：

- `outputs`：在输出中要解释的标量值 -- 要解释什么。例如，在分类的概率输出\[0.1, 0.2, 0.7\]中，如果想要解释0.7，考虑以下公式，其中输出是`y`，这就是我们想要解释的内容。

    y = f(x)

考虑以下公式，其中输出是`y`和`z`。由于我们只能对一个标量值进行归因，我们必须选择是想解释输出`y`还是`z`。假设在这个例子中，模型是目标检测，y和z分别是边界框和对象分类。您会想要选择要解释的两个输出中的哪一个。

    y, z = f(x)

`outputs`的字典格式为：

    { "outputs": { "[your_display_name]":
                   "output_tensor_name": [layer]
                 }
    }

<blockquote>
 - [your_display_name]：您为要解释的输出分配的人类可读名称。一个常见的例子是“概率”。<br/>
 - "output_tensor_name"：用于标识要解释的输出层的键/值字段。<br/>
 - [layer]：要解释的输出层。在单任务模型中，比如表格回归器，它是模型中的最后（最顶层）一层。
</blockquote>

- `inputs`：对归因的特征 -- 它们如何影响输出。考虑以下公式，其中`a`和`b`是特征。我们必须选择哪些特征来解释它们是如何影响输出的。假设该模型用于A/B测试，在预测中，`a`是数据项，而`b`标识模型实例是A还是B。您将会选择`a`（或某些子集）作为特征，而不选择`b`，因为它对预测没有贡献。

    y = f(a, b)

`inputs`的最小字典格式为：

    { "inputs": { "[your_display_name]":
                  "input_tensor_name": [layer]
                 }
    }

<blockquote>
 - [your_display_name]：您为要解释的输入分配的人类可读名称。一个常见的例子是“特征”。<br/>
 - "input_tensor_name"：用于标识特征归因的输入层的键/值字段。<br/>
 - [layer]：特征归因的输入层。在单输入张量模型中，它是模型中的第一（最底层）一层。
</blockquote>

由于模型的输入是表格数据，您可以指定以下两个额外字段作为报告/可视化辅助：

<blockquote>
 - "modality": "image"：指示字段值是图像数据。
</blockquote>

由于模型的输入是图像，您可以指定以下额外字段作为报告/可视化辅助：

<blockquote>
 - "modality": "image"：指示字段值是图像数据。
</blockquote>

In [ ]:
random_baseline = np.random.rand(32, 32, 3)
input_baselines = [{"number_vaue": x} for x in random_baseline]

INPUT_METADATA = {"input_tensor_name": CONCRETE_INPUT, "modality": "image"}

OUTPUT_METADATA = {"output_tensor_name": serving_output}

input_metadata = aip.ExplanationMetadata.InputMetadata(INPUT_METADATA)
output_metadata = aip.ExplanationMetadata.OutputMetadata(OUTPUT_METADATA)

metadata = aip.ExplanationMetadata(
    inputs={"image": input_metadata}, outputs={"class": output_metadata}
)

explanation_spec = aip.ExplanationSpec(metadata=metadata, parameters=parameters)

### 上传模型

使用这个辅助函数`upload_model`来上传您的模型，存储在SavedModel格式中，到`Model`服务中。这将为您的模型实例化一个Vertex `Model`资源实例。一旦您完成了这一步，您可以像对待其他Vertex `Model`资源实例一样使用`Model`资源实例，比如部署到用于提供预测的`Endpoint`资源。

这个辅助函数接受以下参数：

- `display_name`：`Endpoint`服务的可读名称。
- `image_uri`：模型部署的容器镜像。
- `model_uri`：指向我们SavedModel工件的Cloud Storage路径。对于本教程，这是`trainer/task.py`保存模型工件的Cloud Storage位置，我们在变量`MODEL_DIR`中指定了。

这个辅助函数调用`Model`客户端服务的方法`upload_model`，它接受以下参数：

- `parent`：用于`Dataset`、`Model`和`Endpoint`资源的Vertex位置根路径。
- `model`：Vertex `Model`资源实例的规范。

现在让我们更深入地了解Vertex模型规范`model`。这是一个字典对象，包含以下字段：

- `display_name`：`Model`资源的可读名称。
- `metadata_schema_uri`：由于您的模型是没有用Vertex`Dataset`资源构建的，所以将此字段留空(`''`)。
- `artificat_uri`：模型以SavedModel格式存储的Cloud Storage路径。
- `container_spec`：这是将安装在`Endpoint`资源上的Docker容器的规范，从中`Model`资源将提供预测。使用您之前设置的变量`DEPLOY_GPU！= None`来使用GPU；否则只分配CPU。
- `explanation_spec`：这是启用您模型可解释性的规范。

将模型上传到Vertex模型资源会返回一个长时间运行的操作，因为这可能需要一些时间。您可以调用`response.result()`，这是一个同步调用，当Vertex模型资源就绪时将返回。

辅助函数返回对应Vertex模型实例`upload_model_response.model`的完全限定标识符。您将保存这个标识符用于后续步骤的变量`model_to_deploy_id`中。

In [ ]:
IMAGE_URI = DEPLOY_IMAGE


def upload_model(display_name, image_uri, model_uri):

    model = aip.Model(
        display_name=display_name,
        artifact_uri=model_uri,
        metadata_schema_uri="",
        explanation_spec=explanation_spec,
        container_spec={"image_uri": image_uri},
    )

    response = clients["model"].upload_model(parent=PARENT, model=model)
    print("Long running operation:", response.operation.name)
    upload_model_response = response.result(timeout=180)
    print("upload_model_response")
    print(" model:", upload_model_response.model)
    return upload_model_response.model


model_to_deploy_id = upload_model(
    "cifar10-" + TIMESTAMP, IMAGE_URI, model_path_to_deploy
)

获取`Model`资源信息

现在让我们仅获取您的模型的模型信息。使用此辅助函数`get_model`，并使用以下参数：

- `name`：`Model`资源的Vertex唯一标识符。

此辅助函数调用Vertex `Model`客户端服务的方法`get_model`，并使用以下参数：

- `name`：`Model`资源的Vertex唯一标识符。

In [ ]:
def get_model(name):
    response = clients["model"].get_model(name=name)
    print(response)


get_model(model_to_deploy_id)

部署`Model`资源

现在部署训练好的Vertex自定义`Model`资源。 这需要两个步骤：

1. 为部署`Model`资源创建一个`Endpoint`资源。

2. 将`Model`资源部署到`Endpoint`资源。

### 创建一个 `Endpoint` 资源

使用这个辅助函数 `create_endpoint` 来创建一个用于部署模型提供预测的端点，需要以下参数：

- `display_name`: 一个可供人类阅读的 `Endpoint` 资源名称。

这个辅助函数使用端点客户端服务的 `create_endpoint` 方法，该方法接受以下参数：

- `display_name`: 一个可供人类阅读的 `Endpoint` 资源名称。

创建一个 `Endpoint` 资源会返回一个长时间运行的操作，因为为了提供服务可能需要一些时间。您可以调用 `response.result()`，这是一个同步调用，当 Endpoint 资源准备就绪时会返回。辅助函数返回 `Endpoint` 资源的 Vertex 完全合格标识符：`response.name`。

In [ ]:
ENDPOINT_NAME = "cifar10_endpoint-" + TIMESTAMP


def create_endpoint(display_name):
    endpoint = {"display_name": display_name}
    response = clients["endpoint"].create_endpoint(parent=PARENT, endpoint=endpoint)
    print("Long running operation:", response.operation.name)

    result = response.result(timeout=300)
    print("result")
    print(" name:", result.name)
    print(" display_name:", result.display_name)
    print(" description:", result.description)
    print(" labels:", result.labels)
    print(" create_time:", result.create_time)
    print(" update_time:", result.update_time)
    return result


result = create_endpoint(ENDPOINT_NAME)

现在获取您创建的`Endpoint`资源的唯一标识符。

In [ ]:
# The full unique ID for the endpoint
endpoint_id = result.name
# The short numeric ID for the endpoint
endpoint_short_id = endpoint_id.split("/")[-1]

print(endpoint_id)

### 计算实例扩展

处理在线预测请求时，您可以选择多种计算实例扩展的方式：

- 单个实例：在线预测请求在单个计算实例上处理。
  - 将计算实例的最小 (`MIN_NODES`) 和最大 (`MAX_NODES`) 数量设置为一。

- 手动扩展：在线预测请求在您手动指定的固定数量的计算实例上分配。
  - 将计算实例的最小 (`MIN_NODES`) 和最大 (`MAX_NODES`) 数量设置为相同数量的节点。当模型首次部署到实例时，固定数量的计算实例会被预留，并且在线预测请求会均匀分配在这些实例上。

- 自动扩展：在线预测请求在可伸缩数量的计算实例上分配。
  - 设置计算实例的最小 (`MIN_NODES`) 数量，在模型首次部署时进行预留和取消预留，并将最大 (`MAX_NODES`) 计算实例数量设置为根据负载情况进行预留。

计算实例的最小数量对应于字段 `min_replica_count`，最大数量对应于字段 `max_replica_count`，在您的后续部署请求中。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

### 部署 `Model` 资源到 `Endpoint` 资源

使用此辅助函数 `deploy_model` 将模型部署到为提供预测服务而创建的端点，使用以下参数：

- `model`：要从训练管道中上传（部署）的 `Model` 资源的 Vertex 完全限定标识符。
- `deploy_model_display_name`：部署模型的人类可读名称。
- `endpoint`：要将 `Model` 资源部署到的 Vertex 完全限定 `Endpoint` 资源标识符。

辅助函数调用 `Endpoint` 客户端服务的 `deploy_model` 方法，该方法接受以下参数：

- `endpoint`：要将 `Model` 资源部署到的 Vertex 完全限定 `Endpoint` 资源标识符。
- `deployed_model`：部署模型的要求。
- `traffic_split`：去往该模型的端点流量的百分比，指定为一个或多个键/值对的字典。
  - 如果只有一个模型，则指定为 **{ "0": 100 }**，其中 "0" 指代上传的这个模型，100 表示100% 的流量。
  - 如果端点上存在现有模型，需要进行流量分配，则指定为，其中 `model_id` 是要部署到的端点的现有模型的模型 ID。百分比必须累加为100。

           { "0": percent, model_id: percent, ... }

现在让我们深入了解 `deployed_model` 参数。此参数指定为包含最少必需字段的 Python 字典：

- `model`：部署的 (上传) `Model` 资源的 Vertex 完全限定标识符。
- `display_name`：部署模型的人类可读名称。
- `dedicated_resources`：这指的是为提供预测请求而扩展的计算实例（副本）数量。
  - `machine_spec`：要提供的计算实例。使用您之前设置的变量 `DEPLOY_GPU != None` 来使用 GPU；否则只分配了 CPU。
  - `min_replica_count`：初始提供的计算实例数量，您之前设置为变量 `MIN_NODES`。
  - `max_replica_count`：要扩展到的最大计算实例数量，您之前设置为变量 `MAX_NODES`。
- `enable_container_logging`：这启用容器事件的日志记录，例如执行失败（默认情况下容器日志记录是禁用的）。在调试部署时通常启用容器日志记录，然后在部署到生产环境时禁用。

#### 流量分配

现在让我们深入了解 `traffic_split` 参数。此参数指定为一个 Python 字典。一开始可能会有点困惑。让我解释一下，您可以将模型的多个实例部署到一个端点，然后设置每个实例接收多少（百分比）流量。

为什么要这样做？也许您已经在生产中部署了以前的版本 -- 让我们称之为 v1。您在 v2 上得到了更好的模型评估，但在部署到生产环境之前，您不能确定它是否真的更好。因此，在流量分配的情况下，您可能希望将 v2 部署到与 v1 相同的端点，但只分配它的流量比例为 10％。这样，您可以在不干扰大多数用户的情况下监视其表现，直到做出最终决定。

#### 响应

该方法返回一个长时间运行的操作 `response`。我们将通过调用 `response.result()` 同步等待操作完成，这将一直阻塞直到模型部署完成。如果这是首次将模型部署到端点，则可能需要额外几分钟来完成资源的配置。

In [ ]:
DEPLOYED_NAME = "cifar10_deployed-" + TIMESTAMP


def deploy_model(
    model, deployed_model_display_name, endpoint, traffic_split={"0": 100}
):

    if DEPLOY_GPU:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_type": DEPLOY_GPU,
            "accelerator_count": DEPLOY_NGPU,
        }
    else:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_count": 0,
        }

    deployed_model = {
        "model": model,
        "display_name": deployed_model_display_name,
        "dedicated_resources": {
            "min_replica_count": MIN_NODES,
            "max_replica_count": MAX_NODES,
            "machine_spec": machine_spec,
        },
        "enable_container_logging": False,
    }

    response = clients["endpoint"].deploy_model(
        endpoint=endpoint, deployed_model=deployed_model, traffic_split=traffic_split
    )

    print("Long running operation:", response.operation.name)
    result = response.result()
    print("result")
    deployed_model = result.deployed_model
    print(" deployed_model")
    print("  id:", deployed_model.id)
    print("  model:", deployed_model.model)
    print("  display_name:", deployed_model.display_name)
    print("  create_time:", deployed_model.create_time)

    return deployed_model.id


deployed_model_id = deploy_model(model_to_deploy_id, DEPLOYED_NAME, endpoint_id)

发出一个在线预测请求

现在对部署的模型进行在线预测。

获取测试项目

您将从数据集的测试（保留）部分中选择一个示例作为测试项目。

In [ ]:
test_image = x_test[0]
test_label = y_test[0]
print(test_image.shape)

准备请求内容
您将发送 CIFAR10 图像作为经过压缩的 JPG 图像，而不是原始未压缩的字节：

- `cv2.imwrite`：使用OpenCV将未压缩的图像写入磁盘作为压缩的JPEG图像。
 - 将图像数据从\[0,1)范围反向归一化为[0,255)。
 - 将32位浮点值转换为8位无符号整数。
- `tf.io.read_file`：将压缩的JPG图像读入内存作为原始字节。
- `base64.b64encode`：将原始字节编码为Base 64编码的字符串。

In [ ]:
import base64

import cv2

cv2.imwrite("tmp.jpg", (test_image * 255).astype(np.uint8))

bytes = tf.io.read_file("tmp.jpg")
b64str = base64.b64encode(bytes.numpy()).decode("utf-8")

### 发送带有解释请求的预测

好的，现在您有一个测试图像。使用这个辅助函数`explain_image`，它接受以下参数：

- `image`：作为numpy数组的测试图像数据列表。
- `endpoint`：在部署`Model`资源的`Endpoint`资源的Vertex完全限定标识符。
- `parameters_dict`：用于提供额外参数。
- `deployed_model_id`：在端点部署多个模型时，部署模型的Vertex完全限定标识符。否则，如果只部署一个模型，则可以设置为`None`。

该函数使用预测客户端服务并使用以下参数调用`explain`方法：

- `endpoint`：在部署`Model`资源的`Endpoint`资源的Vertex完全限定标识符。
- `instances`：要预测和解释的实例（编码图像）列表。
- `parameters`：用于提供的额外参数。
- `deployed_model_id`：在端点部署多个模型时，部署模型的Vertex完全限定标识符。否则，如果只部署一个模型，则可以设置为`None`。

为了将图像数据传递给预测服务，在先前的步骤中，您将字节编码为base 64 -- 这样在通过网络传输二进制数据时可以确保内容不被修改。您需要告诉提供二进制信息的地方您的模型部署在哪里，以便它知道内容已被base 64编码，因此在接收端会对其进行解码。

预测请求中的每个实例都是一个以下形式的字典条目：

                        {input_name: {'b64': content}}

- `input_name`：底层模型的输入层的名称。
- `'b64'`：指示内容已经被base 64编码的键。
- `content`：压缩的JPG图像字节作为base 64编码字符串。

由于`predict()`服务可以接受多个图像（实例），您将把您的单个图像作为一个图像列表发送。最后一步，您将实例列表打包成Google的protobuf格式 -- 这是我们传递给`explain()`服务的内容。

`response`对象返回一个列表，列表中的每个元素对应请求中的相应图像。您将在每个预测的输出中看到：

- `deployed_model_id` -- 进行预测/解释的模型的Vertex完全限定标识符。
- `predictions` -- 对于十个类别中的每一个，预测的置信水平（`predictions`），介于0和1之间。
- `explanations` -- 每个特征如何影响预测。

In [ ]:
def explain_image(image, endpoint, parameters_dict, deployed_model_id):
    parameters = json_format.ParseDict(parameters_dict, Value())

    # The format of each instance should conform to the deployed model's prediction input schema.
    instances_list = [{serving_input: {"b64": image}}]
    instances = [json_format.ParseDict(s, Value()) for s in instances_list]

    response = clients["prediction"].explain(
        endpoint=endpoint,
        instances=instances,
        parameters=parameters,
        deployed_model_id=deployed_model_id,
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    predictions = response.predictions
    print("predictions")
    for prediction in predictions:
        print(" prediction:", prediction)

    explanations = response.explanations
    print("explanations")
    for explanation in explanations:
        print(" explanation:", explanation)

    return response


response = explain_image(b64str, endpoint_id, None, None)

### 了解解释回答

在不考虑解释的情况下预览图像及其预测的类别。模型为什么会预测这些类别？

In [ ]:
from io import BytesIO

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

CLASSES = [
    "airplane",
    "automobile",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
]

# Note: change the `ig_response` variable below if you didn't deploy an IG model
for prediction in response.predictions:
    label_index = np.argmax(prediction)
    class_name = CLASSES[label_index]
    confidence_score = prediction[label_index]
    print(
        "Predicted class: "
        + class_name
        + "\n"
        + "Confidence score: "
        + str(confidence_score)
    )

    image = base64.b64decode(b64str)
    image = BytesIO(image)
    img = mpimg.imread(image, format="JPG")

    plt.imshow(img, interpolation="nearest")
    plt.show()

### 用AI解释可视化图像

返回的图像显示了模型预测的顶级类别的解释。这意味着如果模型的预测有误，您看到突出显示的像素属于*错误的类别*。例如，如果模型预测为"飞机"，而实际应该预测为"猫"，您可以看到模型将这幅图像分类为飞机的原因。

如果您部署了一个集成梯度模型，您可以可视化其特征归因。当前，AI解释返回的突出显示像素显示了对模型预测做出贡献的前60%的像素。运行下面的单元格后看到的像素是最能信号模型预测的像素。

In [ ]:
import io

for explanation in response.explanations:
    attributions = dict(explanation.attributions[0].feature_attributions)
    label_index = explanation.attributions[0].output_index[0]
    class_name = CLASSES[label_index]
    b64str = attributions["image"]["b64_jpeg"]
    image = base64.b64decode(b64str)
    image = io.BytesIO(image)
    img = mpimg.imread(image, format="JPG")

    plt.imshow(img, interpolation="nearest")
    plt.show()

取消部署`Model`资源

现在从serving`Endpoint`资源中取消部署您的`Model`资源。使用这个辅助函数`undeploy_model`，它需要以下参数：

- `deployed_model_id`：当`Model`资源部署到时由终端服务返回的模型部署标识符。
- `endpoint`：`Model`部署到的`Endpoint`资源的 Vertex 完全限定标识符。

此函数调用端点客户端服务的方法`undeploy_model`，使用以下参数：

- `deployed_model_id`：当`Model`资源部署时由终点服务返回的模型部署标识符。
- `endpoint`：`Model`资源部署到的`Endpoint`资源的 Vertex 完全限定标识符。
- `traffic_split`：如何在`Endpoint`资源上的其余部署模型之间分配流量。

由于这是`Endpoint`资源上唯一部署的模型，您可以通过将`traffic_split`设置为空来简单地留空`traffic_split`。

In [ ]:
def undeploy_model(deployed_model_id, endpoint):
    response = clients["endpoint"].undeploy_model(
        endpoint=endpoint, deployed_model_id=deployed_model_id, traffic_split={}
    )
    print(response)


undeploy_model(deployed_model_id, endpoint_id)

清理

要清理此项目中使用的所有GCP资源，您可以删除用于教程的GCP项目。

否则，您可以删除本教程中创建的个别资源:

- 数据集
- 流水线
- 模型
- 终端
- 批处理作业
- 自定义作业
- 超参数调整作业
- 云存储桶

In [ ]:
delete_dataset = True
delete_pipeline = True
delete_model = True
delete_endpoint = True
delete_batchjob = True
delete_customjob = True
delete_hptjob = True
delete_bucket = True

# Delete the dataset using the Vertex fully qualified identifier for the dataset
try:
    if delete_dataset and "dataset_id" in globals():
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex fully qualified identifier for the pipeline
try:
    if delete_pipeline and "pipeline_id" in globals():
        clients["pipeline"].delete_training_pipeline(name=pipeline_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model and "model_to_deploy_id" in globals():
        clients["model"].delete_model(name=model_to_deploy_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex fully qualified identifier for the endpoint
try:
    if delete_endpoint and "endpoint_id" in globals():
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex fully qualified identifier for the batch job
try:
    if delete_batchjob and "batch_job_id" in globals():
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

# Delete the custom job using the Vertex fully qualified identifier for the custom job
try:
    if delete_customjob and "job_id" in globals():
        clients["job"].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

# Delete the hyperparameter tuning job using the Vertex fully qualified identifier for the hyperparameter tuning job
try:
    if delete_hptjob and "hpt_job_id" in globals():
        clients["job"].delete_hyperparameter_tuning_job(name=hpt_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r $BUCKET_NAME